<a href="https://colab.research.google.com/github/EmilijaSvirsk/DeepLearning-CustomDataset/blob/main/1lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Emilija Svirskytė 2gr, 1 lab, modelis - resnet18, klasės - bee, ice cream, mushroom

# Pirma užduotis reikalaus realizuoti:
  - efektyvią duomenų nuskaitymo programą su pasirinku egzistuojančiu iš anksto apmokytu (angl. pre-trained) vaizdų klasifikavimo modeliu,
  - paskaičiuoti tikslumo, precizijos, atkūrimo ir F1 statistikas pasirinktiems 1000 paveikslėlių iš OpenImages, (kiekvienai klasei atskirai, galima tureti funkcija, kazkuris bus su visom klasme, 1000 pav susumavus visas klases)
  - realizuoti slenkstinės reikšmės (angl. threshold) keitimą, įgalinant klasifikuoti vaizdus kiekvienai užduotai klasei keičiant T=[0,1]
  - Statistikos turi persiskaičiuoti po slenkstinės reikšmės pakeitimo.
  - Studentai patys savo nuožiūra pasirenka tris klases, kurioms skaičiuos statistikas.

In [ ]:
from urllib.request import urlopen
from numba import jit, cuda
from torchvision import datasets
from torchvision.transforms import transforms
from torchvision import models
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch
import numpy as np
from skimage import io
from skimage.transform import resize
from skimage.color import gray2rgb
import glob
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import os
from os.path import exists

In [ ]:
#paveiksliuku laikymo klase
class CustomDataset(Dataset):
    #on initialization
    @jit(target_backend='cuda')
    def __init__(self, images_dir, classes_names):
        self.images_dir = images_dir
        #issaugome direktorijas
        #glob duoda lista visu dir failu
        self.class_files = []
        self.files = []
        self.class_len = []

        for i in range(len(classes_names)):
          self.class_files.append(glob.glob(self.images_dir + "/{}/images/*.jpg".format(classes_names[i])))
          self.files += self.class_files[i]
          self.class_len.append(len(self.class_files[i]))

        #issaugome teisingas klases paveiksliuku
        self.labels = np.empty(len(self.files), dtype=object)
        self.len_new = 0
        self.len_old = 0
        for i in range(len(classes_names)):
          self.len_new += self.class_len[i]
          self.labels[self.len_old:self.len_new] = classes_names[i]
          self.len_old = self.len_new

        #shuffle data
        self.order =  [x for x in np.random.permutation(len(self.labels))]
        self.files = [self.files[x] for x in self.order]
        self.labels = [self.labels[x] for x in self.order]

    def __len__(self):
        return (len(self.labels))

    def __getitem__(self, i):
        files = self.files[i]
        img = image_transform(Image.open(files))
        y = self.labels[i]
        #grazinam pav ir label
        return (img, y)

In [ ]:
def image_transform(img):
    #if grayscale
    if(img.mode!='RGB'):
      img = img.convert('RGB')

    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )

    # define transforms
    transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            normalize,
    ])

    return transform(img)

In [ ]:
#praeinam paveiksliukus su modeliu ir issaugojam gautas vertes
class ModelTest(Dataset):
    #on initialization
    def __init__(self, model, img_data):
        #how many top results are picked
        top_num = 100
        #dataloader parameters
        size = 100
        workers = 2

        self.result_prob_array = []
        self.result_index_array = []
        self.result_corr = []

        data_loader = torch.utils.data.DataLoader(img_data, batch_size = size, num_workers = workers)

        for batch in data_loader:
          img_array, label_array = batch
          with torch.no_grad():
              out = model(img_array)
              for proc, label in zip(out, label_array):
                proc_array = torch.sigmoid(proc)
                index_array = np.argpartition(proc_array, -top_num)[-top_num:]

                self.result_prob_array.append(proc_array[index_array])
                self.result_index_array.append(index_array)
                self.result_corr.append(label)

    def __len__(self):
        return (len(self.result_prob_array))

    def __getitem__(self, i):
        return [self.result_prob_array[i],self.result_index_array[i],self.result_corr[i]]

In [ ]:
#sukuriam database
data_dir = "drive/MyDrive/GMM_images/data"
classes_names = ["bee", "ice cream", "mushroom"]
dataset = CustomDataset(data_dir, classes_names)

<ipython-input-2-2a90de5ae548>:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "__init__" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "<ipython-input-2-2a90de5ae548>", line 19:
    def __init__(self, images_dir, classes_names):
        <source elided>
        #issaugome teisingas klases paveiksliuku
        self.labels = np.empty(len(self.files), dtype=object)
        ^

  @jit(target_backend='cuda')
<ipython-input-2-2a90de5ae548>:4: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "__init__" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-2-2a90de5ae548>", line 13:
    def __init__(self, images_dir, classes_names):
        <source elided>

        for i in range(len(classes_names)):
        ^

  @jit(target_backend='cuda')
/usr/local/

In [ ]:
#užkraunamas jau apmokytas modelis vgg16
model1 = models.vgg19(pretrained = True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
#modelis grazina tikimybes
model_probs = ModelTest(model, dataset)

# Modelio vertinimo metrikos

Iš pradžių reikia paskaičiuoti TP, TN, FP, FN. Tada galima skaičiuoti tikslumą (_accuracy_), atkūrimą (_recall_), preciziškumą (_precision_), F1.

$Accuracy = \frac{TP + TN}{TP + FP + TN + FN}$

$Recall = \frac{TP}{TP + FN}$

$Precision = \frac{TP}{TP + FP}$

$F1 = 2 \cdot \frac{Precision \cdot Recall}{Precision + Recall}$

In [ ]:
def getClassNames():
  if(not(exists("general_classification.txt"))):
    !wget https://www.sentisight.ai/downloads/general_classification.txt

  with open('general_classification.txt', 'r') as fid:
      return [ln.strip() for ln in fid]


In [ ]:
def getConfusionMatrix(ModelTest, img_class_name, model_class_name, threshold):
  TP = 0
  TN = 0
  FP = 0
  FN = 0
  index = getClassNames().index(model_class_name)

  for i in range(len(ModelTest)):
    sample = ModelTest[i]
    prob_array = sample[0]
    index_array = sample[1]
    true_result = sample[2]

    index_result = np.where(index_array == index)[0]
    if(index_result.size > 0):
      prob_result = prob_array[index_result]
    else:
      prob_result = -1

    #TP obj exists and model found it
    if((true_result==img_class_name) and (prob_result>=threshold)):
      TP+=1
    #TN obj doesnt exist and model didnt find it
    elif((true_result!=img_class_name) and (prob_result<threshold)):
      TN+=1
    #FP obj exists but model didnt find it
    elif((true_result==img_class_name) and (prob_result<threshold)):
      FP+=1
    #FN obj doesnt exist but model found it
    elif((true_result!=img_class_name) and (prob_result>=threshold)):
      FN+=1

  return {"TP":TP,"TN":TN,"FP":FP,"FN":FN}



In [ ]:
def getStatistics(matrix):
  TP = matrix['TP']
  TN = matrix['TN']
  FP = matrix['FP']
  FN = matrix['FN']

  stats = {}
  stats['accuracy'] = (TP + TN) / (TP + FP + TN + FN)
  stats['recall'] = TP / (TP + FN)
  stats['precision'] = TP / (TP + FP)
  stats['f1'] = 2 * (stats['precision'] * stats['recall']) / (stats['precision'] + stats['recall'])

  return stats

In [ ]:
active = True
while active:
  print("Input threshold: ")
  threshold = float(input())

  img_class_name = "bee"
  model_class_name = "bee"
  matrix1 = getConfusionMatrix(model_probs, img_class_name, model_class_name, threshold)
  stats = getStatistics(matrix1)
  print("\nbee class: ")
  print(matrix1)
  print(stats)

  img_class_name = "ice cream"
  model_class_name = "ice cream, icecream"
  matrix2 = getConfusionMatrix(model_probs, img_class_name, model_class_name, threshold)
  stats = getStatistics(matrix2)
  print("\nice cream class: ")
  print(matrix2)
  print(stats)

  img_class_name = "mushroom"
  model_class_name = "mushroom"
  matrix3 = getConfusionMatrix(model_probs, img_class_name, model_class_name, threshold)
  stats = getStatistics(matrix3)
  print("\nmushroom class: ")
  print(matrix3)
  print(stats)

  TP = matrix1["TP"]+matrix2["TP"]+matrix3["TP"]
  TN = matrix1["TN"]+matrix2["TN"]+matrix3["TN"]
  FP = matrix1["FP"]+matrix2["FP"]+matrix3["FP"]
  FN = matrix1["FN"]+matrix2["FN"]+matrix3["FN"]
  matrix = {"TP":TP,"TN":TN,"FP":FP,"FN":FN}
  stats = getStatistics(matrix)
  print("\nall classes: ")
  print(matrix)
  print(stats)

Input threshold: 
0.6

bee class: 
{'TP': 293, 'TN': 799, 'FP': 107, 'FN': 1}
{'accuracy': 0.91, 'recall': 0.9965986394557823, 'precision': 0.7325, 'f1': 0.8443804034582133}

ice cream class: 
{'TP': 251, 'TN': 793, 'FP': 149, 'FN': 7}
{'accuracy': 0.87, 'recall': 0.9728682170542635, 'precision': 0.6275, 'f1': 0.762917933130699}

mushroom class: 
{'TP': 263, 'TN': 789, 'FP': 137, 'FN': 11}
{'accuracy': 0.8766666666666667, 'recall': 0.9598540145985401, 'precision': 0.6575, 'f1': 0.7804154302670624}

all classes: 
{'TP': 807, 'TN': 2381, 'FP': 393, 'FN': 19}
{'accuracy': 0.8855555555555555, 'recall': 0.9769975786924939, 'precision': 0.6725, 'f1': 0.7966436327739388}
Input threshold: 
0.99

bee class: 
{'TP': 225, 'TN': 800, 'FP': 175, 'FN': 0}
{'accuracy': 0.8541666666666666, 'recall': 1.0, 'precision': 0.5625, 'f1': 0.72}

ice cream class: 
{'TP': 194, 'TN': 798, 'FP': 206, 'FN': 2}
{'accuracy': 0.8266666666666667, 'recall': 0.9897959183673469, 'precision': 0.485, 'f1': 0.65100671140939

KeyboardInterrupt: ignored